In [4]:
import pandas as pd
import numpy as np

# 1. Đọc file CSV với kiểm tra lỗi
try:
    df = pd.read_csv('patient_heart_rate.csv', header=None, on_bad_lines='skip', skip_blank_lines=True)
    print("Đọc file thành công. Dữ liệu ban đầu:")
    print(df.head())
except FileNotFoundError:
    print("Lỗi: Không tìm thấy file 'patient_heart_rate.csv'. Vui lòng kiểm tra đường dẫn.")
    exit()
except Exception as e:
    print(f"Lỗi khi đọc file: {e}")
    exit()

if df is None or df.empty:
    print("Lỗi: File không chứa dữ liệu hợp lệ.")
    exit()

# Đặt tên cột thủ công
df.columns = ['ID', 'Name', 'Age', 'Weight', 'HR1', 'HR2', 'HR3', 'HR4', 'HR5', 'HR6']

# Chuyển các cột HR sang kiểu số, thay '-' bằng NaN
hr_columns = ['HR1', 'HR2', 'HR3', 'HR4', 'HR5', 'HR6']
for col in hr_columns:
    df[col] = pd.to_numeric(df[col], errors='coerce')  # Chuyển thành số, giá trị không hợp lệ thành NaN

# 2. Tách cột Name thành FirstName và LastName
try:
    df[['FirstName', 'LastName']] = df['Name'].str.split(' ', n=1, expand=True)
    df = df.drop('Name', axis=1)
except Exception as e:
    print(f"Lỗi khi tách cột Name: {e}")
    exit()

# 3. Chuẩn hóa cột Weight về đơn vị kg
try:
    df['WeightValue'] = df['Weight'].str.extract(r'(\d+\.?\d*)').astype(float)
    df['WeightUnit'] = df['Weight'].str.extract(r'(kgs|lbs)')
    df['WeightValue'] = np.where(df['WeightUnit'] == 'lbs', df['WeightValue'] * 0.453592, df['WeightValue'])
    df['Weight'] = df['WeightValue']
    df = df.drop(['WeightValue', 'WeightUnit'], axis=1)
except Exception as e:
    print(f"Lỗi khi chuẩn hóa Weight: {e}")
    exit()

# 4. Xóa các dòng trống (nếu còn sót)
try:
    df = df.dropna(how='all')
    print("Số dòng sau khi xóa dòng trống:", len(df))
except Exception as e:
    print(f"Lỗi khi xóa dòng trống: {e}")
    exit()

# 5. Xóa các dòng trùng lặp
df = df.drop_duplicates(subset=['FirstName', 'LastName', 'Age', 'Weight', 'HR1', 'HR2', 'HR3', 'HR4', 'HR5', 'HR6'])

# 6. Xử lý ký tự non-ASCII
df['FirstName'] = df['FirstName'].str.replace('é', 'e').str.replace('ö', 'o')
df['LastName'] = df['LastName'].str.replace('é', 'e').str.replace('ö', 'o')

# 7. Thống kê dữ liệu thiếu trên cột Age và Weight
print("\nThống kê dữ liệu thiếu:")
print("Age thiếu:", df['Age'].isna().sum())
print("Weight thiếu:", df['Weight'].isna().sum())

# 8. Xử lý dữ liệu thiếu cho Age và Weight
df = df.dropna(subset=['Age', 'Weight'], how='all')
age_mean = df['Age'].mean()
df['Age'] = df['Age'].fillna(age_mean)

# 9. Thống kê tỷ lệ thiếu trên các cột huyết áp
print("\nTỷ lệ dữ liệu thiếu trên các cột huyết áp:")
for col in hr_columns:
    missing_ratio = df[col].isna().sum() / len(df) * 100
    print(f"{col}: {missing_ratio:.2f}%")

# 10. Xử lý dữ liệu thiếu cho các cột huyết áp
def fill_missing_hr(row, col_idx):
    hr_vals = row[hr_columns].values
    if pd.isna(hr_vals[col_idx]):
        if col_idx > 0 and col_idx < len(hr_columns) - 1:
            if not pd.isna(hr_vals[col_idx - 1]) and not pd.isna(hr_vals[col_idx + 1]):
                return (hr_vals[col_idx - 1] + hr_vals[col_idx + 1]) / 2
        if col_idx > 1 and not pd.isna(hr_vals[col_idx - 1]) and not pd.isna(hr_vals[col_idx - 2]):
            return (hr_vals[col_idx - 1] + hr_vals[col_idx - 2]) / 2
        if col_idx < len(hr_columns) - 2 and not pd.isna(hr_vals[col_idx + 1]) and not pd.isna(hr_vals[col_idx + 2]):
            return (hr_vals[col_idx + 1] + hr_vals[col_idx + 2]) / 2
        valid_hrs = [x for x in hr_vals if not pd.isna(x)]
        if valid_hrs:
            return np.mean(valid_hrs)
        return df[hr_columns].mean().mean()
    return hr_vals[col_idx]

for i, col in enumerate(hr_columns):
    df[col] = df.apply(lambda row: fill_missing_hr(row, i), axis=1)

# 11. Reindex và lưu file
df = df.reset_index(drop=True)
df.to_csv('patient_heart_rate_clean.csv', index=False)
print("\nDữ liệu đã được xử lý và lưu vào 'patient_heart_rate_clean.csv'.")
print(df)

Đọc file thành công. Dữ liệu ban đầu:
     0               1     2           3   4   5   6   7    8   9
0  1.0    Mickéy Mousé  56.0       70kgs  72  69  71   -    -   -
1  2.0     Donald Duck  34.0   154.89lbs   -   -   -  85   84  76
2  3.0      Mini Mouse  16.0         NaN   -   -   -  65   69  72
3  4.0  Scrooge McDuck   NaN       78kgs  78  79  72   -    -   -
4  5.0    Pink Panther  54.0  198.658lbs   -   -   -  69  NaN  75
Số dòng sau khi xóa dòng trống: 14

Thống kê dữ liệu thiếu:
Age thiếu: 3
Weight thiếu: 3

Tỷ lệ dữ liệu thiếu trên các cột huyết áp:
HR1: 58.33%
HR2: 58.33%
HR3: 58.33%
HR4: 41.67%
HR5: 50.00%
HR6: 41.67%

Dữ liệu đã được xử lý và lưu vào 'patient_heart_rate_clean.csv'.
      ID   Age     Weight        HR1        HR2        HR3   HR4    HR5  \
0    1.0  56.0  70.000000  72.000000  69.000000  71.000000  70.0  70.50   
1    2.0  34.0  70.256865  81.666667  81.666667  83.333333  85.0  84.00   
2    3.0  16.0        NaN  68.666667  68.666667  66.833333  65.0  69.0